In [47]:
import boto3
import numpy as np
import os
from datetime import date

s3 = boto3.resource('s3')
bucket_name = 'ecms-nuxeo-testing'
my_bucket = s3.Bucket(bucket_name)
files = []
for my_bucket_object in my_bucket.objects.filter(Prefix='deepdetect/'):
    files.append(my_bucket_object)

In [56]:
files = list(filter(lambda x: len(x.key.split('/')) == 3, files))

In [58]:
not_nrmp = list(filter(lambda x: '_NRMP' not in x.key and '-guidance.txt' not in x.key and '-description.txt' not in x.key, files))

In [59]:
sampled = np.random.choice(not_nrmp[1:], 1000, replace=False)
sampled = sorted(sampled,key= lambda x: x.key.split('/')[-1])

In [60]:
today = date.today()
d4 = today.strftime("%m-%d-%Y")

In [5]:
folder_name = 'nrmp-review-' + d4
if not os.path.isdir(folder_name):
    os.mkdir(folder_name)
s3_client = boto3.client('s3')
for obj in sampled:
    bucket = obj.bucket_name
    key = obj.key
    file_name = key.split('/')[-1]
    with open(folder_name + '/' + file_name, 'wb') as f:
        s3_client.download_fileobj(bucket, key, f)

In [6]:
data = []
for obj in sampled:
    bucket = obj.bucket_name
    key = obj.key
    file_name = key.split('/')[-1]
    record_schedule = key.split('/')[-2]
    data.append({'file_name':file_name, 'schedule':record_schedule})

In [11]:
import pandas as pd
df = pd.DataFrame.from_records(data)
df['Action'] = ""
df['Comment'] = ""

In [12]:
names = ['File', 'Schedule', 'Action', 'Comment']
df.to_excel('nrmp-review-sheet-' + d4 + '.xlsx', header=names, index=False)

In [35]:
import shutil
shutil.make_archive('Training Data QA-' + d4, 'zip', folder_name)

'C:\\Users\\mkreisel\\content-ingestion-services\\training_data_generation\\Training Data QA-12-11-2020.zip'

In [18]:
labeled = pd.read_excel('Training Data QA Spreadsheet_11-04-20.xlsx', engine='openpyxl')

In [24]:
current = set(df['file_name'])
old = set(labeled['Filename'])

In [28]:
exclusive = current - old

In [30]:
filtered = df[df['file_name'].isin(exclusive)]

In [31]:
len(filtered)

980

In [42]:
filtered.to_excel('Training Data QA Spreadsheet-' + d4 + '.xlsx', header=names, index=False)

In [40]:
filtered = filtered[filtered['schedule'] != 'deepdetect']